# German Credit Dataset
- 대출인지 아닌지를 예측하는 문제
- 데이터를 NB에 맞도록 간단하게 변환합니다.
- Binary 데이터들로 이루어진 대출 사기 데이터들로 부터 대출인지 아닌지 예측해보세요.

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [2]:
data_url = './fraud.csv'
df = pd.read_csv(data_url, sep=',')
df.head()

,ID,History,CoApplicant,Accommodation,Fraud
0,1,current,none,own,True
1,2,paid,none,own,False
2,3,paid,none,own,False
3,4,paid,guarantor,rent,True
4,5,arrears,none,own,False


In [3]:
# ID열을 삭제해줍니다.
del df["ID"] 

# Label(Y_data)을 따로 저장해 줍니다.
Y_data = df.pop("Fraud")

In [4]:
# as_matrix()함수를 통해 array형태로 변환시켜 줍니다.
# Convert the frame to its Numpy-array representation.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.as_matrix.html

Y_data = Y_data.as_matrix()
Y_data

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


array([ True, False, False,  True, False,  True, False, False, False,
        True, False,  True,  True, False, False, False, False, False,
       False, False])

In [5]:
type(Y_data)

numpy.ndarray

In [6]:
df.head()

,History,CoApplicant,Accommodation
0,current,none,own
1,paid,none,own
2,paid,none,own
3,paid,guarantor,rent
4,arrears,none,own


In [7]:
# 우리가 앞으로 사용할 데이터 셋입니다. 그런데 문제가 있어보이네요...

## One-Hot encoding

* 범주형 변수를 dummy변수로 변환해주는 작업
* Do it yourself!

### 1. Do One-Hot Encoding! 

In [8]:
df.dtypes # 전부 범주형 변수이다.

History          object
CoApplicant      object
Accommodation    object
dtype: object

In [9]:
df.isnull().sum()

History          0
CoApplicant      0
Accommodation    0
dtype: int64

In [10]:
# 범주형 변수 처리 문제입니다.
# 앞선 EDA 시간과 Logistic EDA를 통해 우리는 범주형 변수를 처리해 주는 방법을 배웠습니다.
# get_dummies를 사용해서 One-Hot encoding 처리를 해주세요.
x_df = pd.get_dummies(df).copy()
x_df.head() # dummy변수로 변환

,History_arrears,History_current,History_none,History_paid,CoApplicant_coapplicant,CoApplicant_guarantor,CoApplicant_none,Accommodation_free,Accommodation_own,Accommodation_rent
0,0,1,0,0,0,0,1,0,1,0
1,0,0,0,1,0,0,1,0,1,0
2,0,0,0,1,0,0,1,0,1,0
3,0,0,0,1,0,1,0,0,0,1
4,1,0,0,0,0,0,1,0,1,0


* One-Hot Encoding이 제대로 되었다면 우리는 10개의 Feature를 얻을 수 있습니다.

In [11]:
x_data = x_df.as_matrix()
x_data

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


array([[0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
       [0, 0, 0, 1, 0, 0, 1, 0, 1, 0]], dtype=uint8)

In [12]:
# one-hot encoding을 통해 10개의 Feature를 얻었다.

#### Q1. as_matrix()함수를 통해 우리가 하고자 하는 것은 무엇일까요? 
#### A. naivebayes에서 likelihood를 쉽게 구하기 위함이다! 교집합 사용해서!

In [13]:
Y_data == True # boolean index

array([ True, False, False,  True, False,  True, False, False, False,
        True, False,  True,  True, False, False, False, False, False,
       False, False])

In [14]:
len(set(Y_data))

2

## Naive bayes classifier

* P(Y)
* P(X1, X2, ..., Xn)
* P(Y|X1, X2, X3, ..., Xn)
* P(X1|Y), P(X2|Y), ... P(Xn|Y)
등 우리가 구해야 할 식들에 대한 아이디어가 있어야 합니다.

### P(Y1), P(Y0) 구하기

In [15]:
# P(Y1), P(Y0)
# P(Y1) = count(Y1) / count(Y)

P_Y_True = sum(Y_data==True) / len(Y_data)
P_Y_False = 1 - P_Y_True

P_Y_True, P_Y_False

(0.3, 0.7)

* 이번 튜토리얼에서는 **index를 이용합니다.**
* 이해하기보다는 따라 하면서 음미해보세요.

In [16]:
# y가 1일 경우, y가 0일 경우를 구해줘야 합니다.
# 이번 시간에는 np.where를 사용합니다.
# np.where

In [17]:
ix_Y_True = np.where(Y_data) # Y_data == True인 인덱스를 뽑아줍니다.
ix_Y_False = np.where(Y_data==False)

ix_Y_True, ix_Y_False

((array([ 0,  3,  5,  9, 11, 12], dtype=int64),),
 (array([ 1,  2,  4,  6,  7,  8, 10, 13, 14, 15, 16, 17, 18, 19],
        dtype=int64),))

In [18]:
# np.where을 사용해서 Y가1일 때와 0일 때 각각의 인덱스 값을 얻을 수 있게 되었습니다.

In [19]:
# P(X|Y) = count(X_cap_Y) / count(Y)

### productP(X|Yc) 구하기

* product * P(X|Y1)
* product * P(X|Y2)

In [20]:
x_data[ix_Y_True].sum(axis=0) 

array([1, 3, 1, 1, 0, 1, 5, 0, 4, 2], dtype=uint32)

In [24]:
p_x_y_true = (x_data[ix_Y_True].sum(axis=0) / sum(Y_data==True))
# Q.뒤에 sum(Y_data == True) 필요한가요? # 앞에 식이 P(X_cap_Y1)인 것 같은데...
# A. p(x|y) = p(x교y) / p(y)이기 때문이다. 분모가 둘다 전체 겟수기 때문에 생략하고
# P(X|Y) = count(X_cap_Y) / count(Y) 이런식으로 표현할 수 있다.
p_x_y_false = x_data[ix_Y_False].sum(axis=0) / sum(Y_data==False)
p_x_y_true, p_x_y_false

(array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
        0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333]),
 array([0.42857143, 0.28571429, 0.        , 0.28571429, 0.14285714,
        0.        , 0.85714286, 0.07142857, 0.78571429, 0.14285714]))

In [25]:
# 총 10개의 값에 대해서 확률을 구해준다.

In [26]:
x_test = [0,1,0,0,0,1,0, 0,1,0]


import math

# p_y_true_test = P_Y_True + p_x_y_true.dot(x_test)
# p_y_false_test = P_Y_False + p_x_y_false.dot(x_test)
p_y_true_test = P_Y_True * p_x_y_true.dot(x_test)
p_y_false_test = P_Y_False * p_x_y_false.dot(x_test)

p_y_true_test, p_y_false_test # 값이 1 넘게 나오는 이유는 

(0.39999999999999997, 0.7499999999999999)

In [27]:
p_y_true_test > p_y_false_test

False

## 2. Do Smoothing을 통해 P(Y=1|X)의 확률과 P(Y=0|X)의 확률 값을 비교하세요.
### smoothing과 log처리 해서 문제 해결하기

In [28]:
smoothing_p = 2
clsss_num = 2

In [29]:
smooth_p_x_y_true = (x_data[ix_Y_True].sum(axis=0) + smoothing_p / sum(Y_data==True) + (smoothing_p * clsss_num))
smooth_p_x_y_false = (x_data[ix_Y_False].sum(axis=0) + smoothing_p / sum(Y_data==False) + (smoothing_p * clsss_num))

### 바로 log를 적용하면 확률이 0인값이 있어 -inf값이 생겨서 올바른 계산을 하지 못한다.

In [30]:
p_x_y_true, np.log(p_x_y_true), np.sum(np.log(p_x_y_true))

C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


(array([0.16666667, 0.5       , 0.16666667, 0.16666667, 0.        ,
        0.16666667, 0.83333333, 0.        , 0.66666667, 0.33333333]),
 array([-1.79175947, -0.69314718, -1.79175947, -1.79175947,        -inf,
        -1.79175947, -0.18232156,        -inf, -0.40546511, -1.09861229]),
 -inf)

In [31]:
smooth_p_x_y_true, np.log(smooth_p_x_y_true), np.sum(np.log(smooth_p_x_y_true))

(array([5.33333333, 7.33333333, 5.33333333, 5.33333333, 4.33333333,
        5.33333333, 9.33333333, 4.33333333, 8.33333333, 6.33333333]),
 array([1.67397643, 1.99243016, 1.67397643, 1.67397643, 1.46633707,
        1.67397643, 2.23359222, 1.46633707, 2.12026354, 1.84582669]),
 17.820692484769264)

In [32]:
p_x_y_true.dot(x_test)

1.3333333333333333

In [33]:
from math import log, exp
x_test = [0,1,0,0,0,1,0, 0,1,0]


p_y_true_test = P_Y_True * p_x_y_true.dot(x_test)
p_y_false_test = P_Y_False * p_x_y_false.dot(x_test)

smooth_p_y_true_test = P_Y_True * smooth_p_x_y_true.dot(x_test)
smooth_p_y_false_test = P_Y_False * smooth_p_x_y_false.dot(x_test)

log_p_y_true_test = log(P_Y_True) + np.sum(np.log(p_x_y_true))
log_p_y_false_test = log(P_Y_False) + np.sum(np.log(p_x_y_false))

smooth_log_p_y_true_test = log(P_Y_True) + np.sum(np.log(smooth_p_x_y_true))
smooth_log_p_y_false_test = log(P_Y_False) + np.sum(np.log(smooth_p_x_y_false))

print("              true              ", "false")
print("BASIC :", p_y_true_test, p_y_false_test, p_y_true_test > p_y_false_test)
print("SMOOTHING :",smooth_p_y_true_test, smooth_p_y_false_test, smooth_p_y_true_test > smooth_p_y_false_test)
print("LOG :", log_p_y_true_test, log_p_y_false_test, log_p_y_true_test > log_p_y_false_test)
print("LOG SMOOTHING :", smooth_log_p_y_true_test, smooth_log_p_y_false_test, smooth_log_p_y_true_test > smooth_log_p_y_false_test)

              true               false
BASIC : 0.39999999999999997 0.7499999999999999 False
SMOOTHING : 6.3 19.2 False
LOG : -inf -inf False
LOG SMOOTHING : 16.616719680443328 19.76434372085914 False


C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
C:\Users\USER\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':


# 결과값에 대한 설명과 해석을 달아주세요
#### basic, log, smoothing, log smoothing 총 4가지 경우를 하였는데 일단 결과는 전부 false로 예측하였다.
####  1. smoothing을 적용하니 0값에 가깝던 확률값이 증가하여 높은 수를 가지고 있다.
#### 2. log를 취하면 더하기 연산을 하게되는데 위에서 말한것 처럼 확률이 0인값이 있어 제대로 계산하지 못한다. 그래서 스무딩이 필요하다.
####    또한 곱연산을 더하기 연산으로 바꿔주기때문에 0으로 수렴하던 현상이 사라진다.
#### 3. smoothing 적용후 log연산까지하면 0으로 수렴하던 현상이 더더욱 사라지게 된다.